In [1]:
# Installation of any non native supported libraries

# !pip install beautifulsoup4
# !pip install wget

In [2]:
# Import necessary libraries

import pandas as pd
import requests
from bs4 import BeautifulSoup
import wget
import os

In [14]:
# Your local download directory
download_destination = '/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data'

## Download Data

In [19]:
r = requests.get(URL) 
with open(r'/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/file_name.xlsx', 'wb') as f:
    f.write(r.content)

In [47]:
# Import openyxl module
import openpyxl
import pandas as pd

# Define variable to load the wookbook
wb = openpyxl.load_workbook("/Users/dorin/Documents/Jupyter Notebook Files/CCT - Masters/MSc_Data_Analytics_CA2/Data/file_name.xlsx")
sheets = wb.sheetnames
ws = wb[sheets[4]]
df = pd.DataFrame(ws.values)

df.head(50)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,PRICES,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Table 3: Price Indices of Construction Materia...,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,United Kingdom,None,None,None,None,None,None,2015=100,None,None,None,None,None,None,None
5,None,None,2018,2019,2020,2021,2022,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,,None,None,None,None,None,None,None,None
7,AGGREGATES,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,"Gravel, Sand, Clays & Kaolin - exc Aggregate ...",None,c,c,c,114.391667,134.458333,None,None,None,None,None,None,None,None
9,...,None,c,116.125,119.341667,126.383333,152.683333,None,None,None,None,None,None,None,None
